In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.text import *

### Load the prediction model and test

In [3]:
path = untar_data(URLs.IMDB)

In [4]:
bs = 32

In [5]:
data_clas = load_data(path, 'data_clas.pkl', bs=bs)

In [6]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)

In [7]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (25000 items)
x: TextList
xxbos xxmaj first things first , this movie is achingly beautiful . a someone who works on xxup 3d xxup cg films as a lighter / compositor , the visuals blew me away . xxmaj every second i was stunned by what was on screen xxmaj as for the story , well , it 's okay . xxmaj it 's not going to set the world on fire , but if you like your futuristic xxmaj blade xxmaj runner - esquire tales ( and who does n't ? ) then you will be fine . 
 
  i do have to say that i felt the voice acting was particularly bland and detracted from the movie as a whole . i saw it at the cinema in xxmaj english , but i am hoping that there is a xxmaj french version floating around somewhere . 
 
  xxmaj definitely worth seeing .,xxbos i just accidentally stumbled over this film on xxup tv one day . xxmaj it was aired in the middle of the day on a channel not exactly famous for airing good movies . xxmaj this one , however , was nothi

In [8]:
learn.load('third')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (25000 items)
x: TextList
xxbos xxmaj first things first , this movie is achingly beautiful . a someone who works on xxup 3d xxup cg films as a lighter / compositor , the visuals blew me away . xxmaj every second i was stunned by what was on screen xxmaj as for the story , well , it 's okay . xxmaj it 's not going to set the world on fire , but if you like your futuristic xxmaj blade xxmaj runner - esquire tales ( and who does n't ? ) then you will be fine . 
 
  i do have to say that i felt the voice acting was particularly bland and detracted from the movie as a whole . i saw it at the cinema in xxmaj english , but i am hoping that there is a xxmaj french version floating around somewhere . 
 
  xxmaj definitely worth seeing .,xxbos i just accidentally stumbled over this film on xxup tv one day . xxmaj it was aired in the middle of the day on a channel not exactly famous for airing good movies . xxmaj this one , however , was nothi

In [9]:
learn.predict('mindhunter is awesome')

(Category pos, tensor(1), tensor([0.2062, 0.7938]))

### Process some tweets

In [49]:
import pathlib
import json
import numpy as np
import pandas as pd
from IPython.display import display

In [11]:
tweet_path = pathlib.Path('/home/jupyter/my_projects/tv_tweets/')

In [13]:
programs = [{'name':'mindhunter','file':'mindhunter_tweets.json'},
           {'name':'designated survivor','file':'des_sur_tweets.json'},
           {'name':'the i-land','file':'i_land_tweets.json'},
           {'name':'another life','file':'al_tweets.json'},]

In [52]:
tweet_file = tweet_path / 'al_tweets.json'

In [53]:
with open(tweet_file,'r') as f:
    tweets = json.load(f)

In [54]:
len(tweets)

100

In [55]:
for t in tweets[0:5]:
    print(t)
    print(learn.predict(t))

recently watched Another Life on @netflix and i really liked it. i dont get all the hate. Sure its not Star Wars, b… https://t.co/7y6bWiz8pk
(Category pos, tensor(1), tensor([0.0227, 0.9773]))
@jasonrdavis I’m afraid for a channel heavy in sci fi, most of the Netflix shows are shite. ST Discovery rides a co… https://t.co/DzxI71PQXx
(Category neg, tensor(0), tensor([0.6372, 0.3628]))
RT @RedFiddler: I just tried to watch Another Life on Netflix. In the first episode, the female protagonist - the captain of an intergalact…
(Category neg, tensor(0), tensor([0.7742, 0.2258]))
#Netflix’s IN THE TALL GRASS Trailer Brings Another #StephenKing Tale To Life

#IntheTallGrass https://t.co/eRLgKyyWyU
(Category pos, tensor(1), tensor([0.2441, 0.7559]))
I just tried to watch Another Life on Netflix. In the first episode, the female protagonist - the captain of an int… https://t.co/D9D7KTNMpE
(Category pos, tensor(1), tensor([0.4383, 0.5617]))


In [35]:
def get_scores(tweets):
    
    cats = []
    scores = []
    
    for tweet in tweets:
        cat, ten, score = learn.predict(tweet)
    
        cats.append(cat.__str__())
        scores.append(score[ten].item())
        
    return cats, scores
    
    

In [36]:
cats, scores = get_scores(tweets)

In [43]:
df = pd.DataFrame(data={'Categories':cats,'Scores':scores})
df.head()

,Categories,Scores
0,neg,0.696478
1,neg,0.828244
2,pos,0.679582
3,pos,0.542215
4,neg,0.507333


In [47]:
df.groupby('Categories').agg({'Categories':len,'Scores':np.mean})

,Categories,Scores
Categories,,
neg,39,0.674914
pos,61,0.721752


In [50]:
for program in programs:
    print(program['name'])
    
    #load tweets
    tweet_file = tweet_path / program['file']
    
    with open(tweet_file,'r') as f:
        tweets = json.load(f)
    
    cats, scores = get_scores(tweets)
    df = pd.DataFrame(data={'Categories':cats,'Scores':scores})
    display(df.groupby('Categories').agg({'Categories':len,'Scores':np.mean}))
    

mindhunter


,Categories,Scores
Categories,,
neg,39,0.674914
pos,61,0.721752


designated survivor


,Categories,Scores
Categories,,
neg,35,0.714103
pos,65,0.755773


the i-land


,Categories,Scores
Categories,,
neg,59,0.772427
pos,41,0.763920


another life


,Categories,Scores
Categories,,
neg,31,0.761778
pos,69,0.783529
